# DATA512 - Homework 2: Considering Bias in Data

## Coverage and Quality Analysis of US Cities in Wikipedia Articles

### Tanushree Yandra, University of Washington, Seattle

The aim of this assignment is to explore the concept of bias in data using Wikipedia articles. The articles considered for this study are about cities in different US states. For this assignment, a dataset of Wikipedia articles is combined with a dataset of state populations, and a machine learning service called ORES is used to estimate the quality of the articles about the cities. An analysis is then conducted on how the coverage of US cities on Wikipedia and how the quality of articles about cities varies among states.

## 1. Data Acquisition

The first step of the assigment is to collect the data. The data of Wikipedia articles about US cities and that of US state populations is required for this study.

For the former, the Wikipedia ['Category:Lists of cities in the United States by state'](https://en.wikipedia.org/wiki/Category:Lists_of_cities_in_the_United_States_by_state) is crawled to generate a list of Wikipedia article pages about US cities from each state. This data can be found in [us_cities_by_state_SEPT.2023.csv](https://drive.google.com/file/d/1khouDmMaZyKo0y5WkFj4lu7g8o35x_98/view?usp=sharing).

For the latter, the US Census Bureau provides updated population estimates for every US state. The data can be found on ['State Population Totals and Components of Change: 2020-2022'](https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html) from their website. An Excel file linked to that page contains estimated populations of all US states for 2022. 

Finally, the 'region' demarcation within the US is not standardized and fixed. In fact, different US government agencies agglomerate states to define regions as a function of differing goals. For this analysis, we will use the regional and divisional agglomerations as defined by the US Census Bureau. The data for the same can be found in ['US States by Region - US Census Bureau'](https://docs.google.com/spreadsheets/d/14Sjfd_u_7N9SSyQ7bmxfebF_2XpR8QamvmNntKDIQB0/edit?usp=sharing).

In [1]:
# These are standard python modules
import json, time, urllib.parse
import pandas as pd
import warnings

# The modules below are not standard Python modules
# You will need to install these with pip/pip3 if you do not already have it
import requests

In [2]:
# Suppress the warning statements
warnings.filterwarnings("ignore")

In [3]:
# Pull the page_titles data from the below csv file
us_cities_df = pd.read_csv("us_cities_by_state_SEPT.2023.csv")
us_cities_df.head()

,state,page_title,url
0,Alabama,"Abbeville, Alabama","https://en.wikipedia.org/wiki/Abbeville,_Alabama"
1,Alabama,"Adamsville, Alabama","https://en.wikipedia.org/wiki/Adamsville,_Alabama"
2,Alabama,"Addison, Alabama","https://en.wikipedia.org/wiki/Addison,_Alabama"
3,Alabama,"Akron, Alabama","https://en.wikipedia.org/wiki/Akron,_Alabama"
4,Alabama,"Alabaster, Alabama","https://en.wikipedia.org/wiki/Alabaster,_Alabama"


In [4]:
us_cities_df.shape

(22157, 3)

In [5]:
# Drop any duplicate columns
us_cities_df.drop_duplicates(inplace=True)

In [6]:
us_cities_df.shape

(21525, 3)

It can be observed that quite a number of duplicates existed and the number of rows have dropped.

Skimming through the data further, it can be found that a few 'page_title' column values are not city names. Such values have been identified and the rows with these values are dropped.

In [7]:
# Miscellaneous page_title values that are not cities and need to be dropped
page_titles_to_remove = ['2020 United States census',
                         '2010 United States census',
                         'County (United States)',
                         'Federal Information Processing Standards',
                         'American National Standards Institute',
                         'Geographic Names Information System',
                         'Wikipedia:Citation needed',
                         'Population',
                         'Square mile']

In [8]:
# Dropping page titles that need to be removed
us_cities_df = us_cities_df[~us_cities_df['page_title'].isin(page_titles_to_remove)]

In [9]:
# Storing the final list of page titles in a list 
cities_titles = us_cities_df['page_title'].to_list()

In [10]:
len(cities_titles)

21510

It is also important to note that some page titles are not in the expected format of 'City, State'. And some values are in the format of 'X, Y' but are still missing the state name in the page title. Such values that are missing the state name in the page title have been stored in a dataframe called the 'filtered_df'. This dataframe will be used later in the data preprocessing step to map the state names.

In [11]:
# Page titles that are not in the format 'X, Y'
page_titles_anomaly = us_cities_df[~us_cities_df['page_title'].str.contains(',', na=False)]
page_titles_anomaly

,state,page_title,url
1062,Alaska,Utqiaġvik,https://en.wikipedia.org/wiki/Utqiaġvik
1901,California,Los Angeles,https://en.wikipedia.org/wiki/Los_Angeles
2029,California,San Diego,https://en.wikipedia.org/wiki/San_Diego
2032,California,San Francisco,https://en.wikipedia.org/wiki/San_Francisco
2211,Colorado,Denver,https://en.wikipedia.org/wiki/Denver
2756,Florida,Miami,https://en.wikipedia.org/wiki/Miami
2953,Georgia_(U.S._state),Atlanta,https://en.wikipedia.org/wiki/Atlanta
3094,Georgia_(U.S._state),Echols County,https://en.wikipedia.org/wiki/Echols_County
5114,Indiana,Indianapolis,https://en.wikipedia.org/wiki/Indianapolis
7409,Louisiana,New Orleans,https://en.wikipedia.org/wiki/New_Orleans


In [12]:
# Page titles that are in the format 'X, Y' but are still missing state names
titles_list = ['Eastwood, Syracuse', 'Wailua, Kauai', 'Hyde Park, Boston', 'Riverview, St. Louis']
page_titles_anomaly2 = us_cities_df[us_cities_df['page_title'].isin(titles_list)]
page_titles_anomaly2

,state,page_title,url
3553,Hawaii,"Wailua, Kauai","https://en.wikipedia.org/wiki/Wailua,_Kauai"
8501,Massachusetts,"Hyde Park, Boston","https://en.wikipedia.org/wiki/Hyde_Park,_Boston"
12175,Missouri,"Riverview, St. Louis","https://en.wikipedia.org/wiki/Riverview,_St._L..."
14045,New_York,"Eastwood, Syracuse","https://en.wikipedia.org/wiki/Eastwood,_Syracuse"


In [13]:
filtered_df = pd.concat([page_titles_anomaly, page_titles_anomaly2])
filtered_df

,state,page_title,url
1062,Alaska,Utqiaġvik,https://en.wikipedia.org/wiki/Utqiaġvik
1901,California,Los Angeles,https://en.wikipedia.org/wiki/Los_Angeles
2029,California,San Diego,https://en.wikipedia.org/wiki/San_Diego
2032,California,San Francisco,https://en.wikipedia.org/wiki/San_Francisco
2211,Colorado,Denver,https://en.wikipedia.org/wiki/Denver
2756,Florida,Miami,https://en.wikipedia.org/wiki/Miami
2953,Georgia_(U.S._state),Atlanta,https://en.wikipedia.org/wiki/Atlanta
3094,Georgia_(U.S._state),Echols County,https://en.wikipedia.org/wiki/Echols_County
5114,Indiana,Indianapolis,https://en.wikipedia.org/wiki/Indianapolis
7409,Louisiana,New Orleans,https://en.wikipedia.org/wiki/New_Orleans


### (a) Page Info Request

The code below illustrates how to access page info data using the [MediaWiki REST API for the EN Wikipedia](https://www.mediawiki.org/wiki/API:Main_page). It shows how to request summary 'page info' for a single article page. The API documentation, [API:Info](https://www.mediawiki.org/wiki/API:Info), covers additional details that may be helpful when trying to use or understand the explanation. The code provided below has been sampled from [this notebook](https://drive.google.com/file/d/15UoE16s-IccCTOXREjU3xDIz07tlpyrl/view?usp=sharing) under the [CC-BY license](https://creativecommons.org/licenses/by/4.0/).

Some constants have been defined below to make the code more readable.

In [14]:
# CONSTANTS

# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"

# We'll assume that there needs to be some throttling for these requests
# We should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming a small delay of roughly 2ms to each request
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
REQUEST_HEADERS = {
    'User-Agent': 'tyandra@uw.edu, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

# Assigning the titles that we retrived earlier
ARTICLE_TITLES = cities_titles

# This is a string of additional page properties that can be returned to see the Info documentation for
# what can be included. If you don't want any this can simply be an empty string
PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"

# This template lists the basic parameters
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",           # To simplify, this should be a single page title at a time
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}

The API request will be made using the below procedure. The procedure is parameterized, but relies on the constants above for the important parameters. The underlying assumption is that this will be used to request data for a set of article pages. Therefore, the parameter most likely to change is the article_title.

In [15]:
# PROCEDURES/FUNCTIONS

def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    
    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['titles'] = article_title

    if not request_template['titles']:
        raise Exception("Must supply an article title to make a pageinfo request.")

    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


In [18]:
title_revid = {}

# Iterate through every article and get the json response
for article in ARTICLE_TITLES:
    pageinfo = request_pageinfo_per_article(article)
    info = pageinfo['query']['pages']
    for key, value in info.items():
        # Extract the revision ID and title from the json response and store it in a dicitionary
        if 'lastrevid' in value:
            lastrevid_value = value['lastrevid']
        if 'title' in value:
            article_title = value['title']
        title_revid[article_title] = lastrevid_value

In [21]:
len(title_revid)

21510

In [22]:
# Save the dictionary in a json file
with open('title_revid.json', 'w') as json_file:
    json.dump(title_revid, json_file, indent=4)

While the data can be accessed by using the dictionary created earlier, the below code shows how to access the data from the JSON file that we just exported.

In [23]:
with open('title_revid.json', 'r') as json_file:
    data_revid = json.load(json_file)

### (b) ORES Request

The next step is to get the predicted quality scores for each article in the Wikipedia dataset. The machine learning system used for this purpose is called [ORES](https://www.mediawiki.org/wiki/ORES). This was originally an acronym for 'Objective Revision Evaluation Service' but was simply renamed 'ORES'. ORES is a machine learning tool that can provide estimates of Wikipedia article quality. The article quality estimates are labelings learned based on articles in Wikipedia that were peer-reviewed using the [Wikipedia content assessment](https://en.wikipedia.org/wiki/Wikipedia:Content_assessment) procedures. These quality classes are a subset of quality assessment categories developed by Wikipedia editors, ranked below from best to worst:

1. FA - Featured article
2. GA - Good article (sometimes called A-class)
3. B - B-class article
4. C - C-class article
5. Start - Start-class article
6. Stub - Stub-class article

ORES requires a specific revision ID of a specific article to be able to make a label prediction. You can use the [API:Info](https://www.mediawiki.org/wiki/API:Info) request to get a range of metadata on an article, including the most current revision ID of the article page. For more information, the [ORES API documentation](https://ores.wikimedia.org/) can be accessed from the main ORES page.

Wikimedia is implementing a new Machine Learning (ML) service infrastructure that they call [LiftWing](https://wikitech.wikimedia.org/wiki/Machine_Learning/LiftWing). Given that ORES already has several ML models that have been well used, ORES is the first set of APIs that are being moved to LiftWing. This code below illustrates how to generate article quality estimates for article revisions using the LiftWing version of ORES. The [ORES LiftWing documentation](https://wikitech.wikimedia.org/wiki/Machine_Learning/LiftWing/Usage) has information about some parameters that have been renamed (e.g., "revid" in the old ORES API is now "rev_id" in the LiftWing ORES API).

The code to request ORES scores through LiftWing ML Service API has been sampled from [this notebook](https://drive.google.com/file/d/17C9xsmR9U3lJeD52UTbAedlHDetwYsxs/view?usp=sharing) under the [CC-BY license](https://creativecommons.org/licenses/by/4.0/).

In [24]:
# CONSTANTS

# The current LiftWing ORES API endpoint and prediction model
API_ORES_LIFTWING_ENDPOINT = "https://api.wikimedia.org/service/lw/inference/v1/models/{model_name}:predict"
API_ORES_EN_QUALITY_MODEL = "enwiki-articlequality"

# The throttling rate is a function of the Access token that you are granted when you request the token
# The constants come from dissecting the token and getting the rate limits from the granted token
# An example of that is below.
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (60.0/5000.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# Because all LiftWing API requests require some form of authentication, access token is needed too
REQUEST_HEADER_TEMPLATE = {
    'User-Agent': "tyandra@uw.edu, University of Washington, MSDS DATA 512 - AUTUMN 2023",
    'Content-Type': 'application/json',
    'Authorization': "Bearer {access_token}"
}

# This is a template for the parameters that we need to supply in the headers of an API request
REQUEST_HEADER_PARAMS_TEMPLATE = {
    'email_address' : "",         # your email address should go here
    'access_token'  : ""          # the access token you create will need to go here
}

# Storing the dictionary of article titles and corresponding revision ids
ARTICLE_REVISIONS = data_revid

# This is a template of the data required as a payload when making a scoring request of the ORES model
ORES_REQUEST_DATA_TEMPLATE = {
    "lang":        "en",     # required that it's english - we're scoring English Wikipedia revisions
    "rev_id":      "",       # this request requires a revision id
    "features":    True
}

# These are used later - defined here so they, at least, have empty values
USERNAME = ""
ACCESS_TOKEN = ""

### Getting your Access Token:

You will need a Wikimedia user account to get access to Lift Wing (the ML API service). You can either [create an account or login](https://api.wikimedia.org/w/index.php?title=Special:UserLogin). If you have a Wikipedia user account - you might already have an Wikimedia account. If you are not sure try your Wikipedia username and password to check it. If you do not have a Wikimedia account you will need to create an account that you can use to get an access token.

There is [a 'guide' that describes how to get authentication tokens](https://api.wikimedia.org/wiki/Authentication) - but not everything works the way it is described in that documentation. You should review that documentation and then read the rest of this comment.

The documentation talks about using a 'dashboard' for managing authentication tokens. You might have a hard time finding this 'dashboard'. First, on the left hand side of the page, you'll see a column of links. The bottom section is a set of links titled "Tools". In that section is a link that says [Special pages](https://api.wikimedia.org/wiki/Special:SpecialPages). At the very bottom of the 'Special pages' page is a section titled 'Other special pages' (scroll all the way to the bottom). The first link in that section is called [API keys](https://api.wikimedia.org/wiki/Special:AppManagement). When you get to the 'API keys' page you can create a new key.

The authentication guide suggests that you should create a server-side app key which did not work when I tried. But, there is an option to create a [Personal API token](https://api.wikimedia.org/wiki/Authentication) that should work for this study and the type of ORES page scoring that you will need to perform.

Note, when you create a Personal API token you are granted the three items - a Client ID, a Client secret, and a Access token - you shold save all three of these. When you dismiss the box they are gone. If you lose any one of the tokens you can destroy or deactivate the Personal API token from the dashboard and then create a new one.

The value you need to work the code below is the Access token - a very long string.

In [25]:
# After following the above steps, you should have a Client ID, a Client secret, and an Access token

# In this case I don't want to distribute my keys with the source of the notebook
# So I wrote a key manager object that helps track all of my API keys
# The key manager hides the keys on disk separate from the code
# A common code idiom to hide API keys will use code to extract the key from an OS environment variable. 
from apikeys.Manager import Manager
keyman = Manager()

USERNAME = "Tanushree Yandra"
key_info = keyman.findRecord(USERNAME,API_ORES_LIFTWING_ENDPOINT)
ACCESS_TOKEN = key_info[0]['key']
#print(ACCESS_TOKEN)

# You can specify these as constants for your own use
# Just don't distribute the notebook without removing your token
#USERNAME = "<your_wikimedia_username>"
#ACCESS_TOKEN = "<your_wikimedia_provided_access_token_its_a_really_long_string>"

The API request will be made using a function to encapsulate call and make access reusable in other notebooks. The procedure is parameterized, relying on the constants above for some important default parameters. The primary assumption is that this function will be used to request data for a set of article revisions. The main parameter is 'article_revid'. One should be able to simply pass in a new article revision id on each call and get back a python dictionary as the result. A valid result will be a dictionary that contains the probabilities that the specific revision is one of six different article quality levels. Generally, quality level with the highest probability score is considered the quality level for the article. This can be tricky when you have two (or more) highly probable quality levels.

In [26]:
# PROCEDURES/FUNCTIONS

def request_ores_score_per_article(article_revid = None, email_address=None, access_token=None,
                                   endpoint_url = API_ORES_LIFTWING_ENDPOINT, 
                                   model_name = API_ORES_EN_QUALITY_MODEL, 
                                   request_data = ORES_REQUEST_DATA_TEMPLATE, 
                                   header_format = REQUEST_HEADER_TEMPLATE, 
                                   header_params = REQUEST_HEADER_PARAMS_TEMPLATE):
    
    # Make sure we have an article revision id, email and token
    # This approach prioritizes the parameters passed in when making the call
    if article_revid:
        request_data['rev_id'] = article_revid
    if email_address:
        header_params['email_address'] = email_address
    if access_token:
        header_params['access_token'] = access_token
    
    # Making a request requires a revision id - an email address - and the access token
    if not request_data['rev_id']:
        raise Exception("Must provide an article revision id (rev_id) to score articles")
    if not header_params['email_address']:
        raise Exception("Must provide an 'email_address' value")
    if not header_params['access_token']:
        raise Exception("Must provide an 'access_token' value")
    
    # Create the request URL with the specified model parameter - default is a article quality score request
    request_url = endpoint_url.format(model_name=model_name)
    
    # Create a compliant request header from the template and the supplied parameters
    headers = dict()
    for key in header_format.keys():
        headers[str(key)] = header_format[key].format(**header_params)
    
    # Make the request
    try:
        # Wait first, to make sure we don't exceed the limit in case an exception occurs while requesting
        # Throttling is always a good practice with a free data source like ORES
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.post(request_url, headers=headers, data=json.dumps(request_data))
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


In [28]:
# Create two lists - one to store the articles with score, and one without
scores_articles = []
no_scores_articles = []

# Iterate through every article
for article_title in ARTICLE_REVISIONS:
    # Call the API to get the score for every article based on the revision id
    score = request_ores_score_per_article(article_revid=ARTICLE_REVISIONS[article_title],
                                           email_address="tyandra@uw.edu",
                                           access_token=ACCESS_TOKEN)
    if score:
        # Create a new dict to store the article title and score
        scores_dict = {}
        string_article = str(ARTICLE_REVISIONS[article_title])
        quality = score["enwiki"]["scores"][string_article]["articlequality"]["score"]["prediction"]
        scores_dict['article_title'] = article_title
        scores_dict['article_quality'] = quality
        # Append the dict to the list
        scores_articles.append(scores_dict)
    else:
        no_scores_articles.append(article_title)


In [29]:
no_scores_articles

[]

No articles were found that were missing a quality score.

In [31]:
scores_articles_df = pd.DataFrame(scores_articles)
# Export the articles and their scores to a csv file
scores_articles_df.to_csv('articles_scores.csv', index=False)

In [32]:
scores_articles_df

,article_title,article_quality
0,"Abbeville, Alabama",C
1,"Adamsville, Alabama",C
2,"Addison, Alabama",C
3,"Akron, Alabama",GA
4,"Alabaster, Alabama",C
...,...,...
21505,"Wamsutter, Wyoming",GA
21506,"Wheatland, Wyoming",GA
21507,"Worland, Wyoming",GA
21508,"Wright, Wyoming",GA


Now we need to merge the revision ids from the data_revid JSON object to the scores_articles_dataframe.

In [33]:
# Convert the data_revid JSON object to a dataframe
data_revid_df = pd.DataFrame(list(data_revid.items()), columns=['article_title', 'revision_id'])
data_revid_df

,article_title,revision_id
0,"Abbeville, Alabama",1171163550
1,"Adamsville, Alabama",1177621427
2,"Addison, Alabama",1168359898
3,"Akron, Alabama",1165909508
4,"Alabaster, Alabama",1179139816
...,...,...
21505,"Wamsutter, Wyoming",1169591845
21506,"Wheatland, Wyoming",1176370621
21507,"Worland, Wyoming",1166347917
21508,"Wright, Wyoming",1166334449


In [34]:
# Merge the data_revid and the scores_articles dataframes
wikipedia_df = pd.merge(data_revid_df, scores_articles_df, on='article_title', how='left')
wikipedia_df

,article_title,revision_id,article_quality
0,"Abbeville, Alabama",1171163550,C
1,"Adamsville, Alabama",1177621427,C
2,"Addison, Alabama",1168359898,C
3,"Akron, Alabama",1165909508,GA
4,"Alabaster, Alabama",1179139816,C
...,...,...,...
21505,"Wamsutter, Wyoming",1169591845,GA
21506,"Wheatland, Wyoming",1176370621,GA
21507,"Worland, Wyoming",1166347917,GA
21508,"Wright, Wyoming",1166334449,GA


From the analysis during the data acquisition step, we know that all the article_titles are not containing the state name. Some article titles are missing the state names. We will now use the 'filtered_df' we created earlier to map the state names for these article_titles.

In [35]:
filtered_df

,state,page_title,url
1062,Alaska,Utqiaġvik,https://en.wikipedia.org/wiki/Utqiaġvik
1901,California,Los Angeles,https://en.wikipedia.org/wiki/Los_Angeles
2029,California,San Diego,https://en.wikipedia.org/wiki/San_Diego
2032,California,San Francisco,https://en.wikipedia.org/wiki/San_Francisco
2211,Colorado,Denver,https://en.wikipedia.org/wiki/Denver
2756,Florida,Miami,https://en.wikipedia.org/wiki/Miami
2953,Georgia_(U.S._state),Atlanta,https://en.wikipedia.org/wiki/Atlanta
3094,Georgia_(U.S._state),Echols County,https://en.wikipedia.org/wiki/Echols_County
5114,Indiana,Indianapolis,https://en.wikipedia.org/wiki/Indianapolis
7409,Louisiana,New Orleans,https://en.wikipedia.org/wiki/New_Orleans


In [36]:
# Make Georgia state name consistent with the Wikipedia dataframe
filtered_df['state'] = filtered_df['state'].str.replace('Georgia_(U.S._state)', 'Georgia')
# Replace any underscores in the dataframe with space
filtered_df['state'] = filtered_df['state'].str.replace('_', ' ')
# Concatenate both the page title and the state string to get the updated page title
filtered_df['updated_page_title'] = filtered_df['page_title'] + ', ' + filtered_df['state']
filtered_df

,state,page_title,url,updated_page_title
1062,Alaska,Utqiaġvik,https://en.wikipedia.org/wiki/Utqiaġvik,"Utqiaġvik, Alaska"
1901,California,Los Angeles,https://en.wikipedia.org/wiki/Los_Angeles,"Los Angeles, California"
2029,California,San Diego,https://en.wikipedia.org/wiki/San_Diego,"San Diego, California"
2032,California,San Francisco,https://en.wikipedia.org/wiki/San_Francisco,"San Francisco, California"
2211,Colorado,Denver,https://en.wikipedia.org/wiki/Denver,"Denver, Colorado"
2756,Florida,Miami,https://en.wikipedia.org/wiki/Miami,"Miami, Florida"
2953,Georgia,Atlanta,https://en.wikipedia.org/wiki/Atlanta,"Atlanta, Georgia"
3094,Georgia,Echols County,https://en.wikipedia.org/wiki/Echols_County,"Echols County, Georgia"
5114,Indiana,Indianapolis,https://en.wikipedia.org/wiki/Indianapolis,"Indianapolis, Indiana"
7409,Louisiana,New Orleans,https://en.wikipedia.org/wiki/New_Orleans,"New Orleans, Louisiana"


In [37]:
# Map the old page titles with the new page titles in a dicitonary
old_titles = filtered_df['page_title'].to_list()
new_titles = filtered_df['updated_page_title'].to_list()
titles_mapping = dict(zip(old_titles, new_titles))

In [38]:
# Use the mapping to replace the page titles in the wikipedia dataframe
wikipedia_df['article_title'] = wikipedia_df['article_title'].replace(titles_mapping)

In [39]:
# Now that all page titles have a state name, extract the name into a new column called 'state'
wikipedia_df['state'] = wikipedia_df['article_title'].str.split(',').str[-1].str.strip()
wikipedia_df

,article_title,revision_id,article_quality,state
0,"Abbeville, Alabama",1171163550,C,Alabama
1,"Adamsville, Alabama",1177621427,C,Alabama
2,"Addison, Alabama",1168359898,C,Alabama
3,"Akron, Alabama",1165909508,GA,Alabama
4,"Alabaster, Alabama",1179139816,C,Alabama
...,...,...,...,...
21505,"Wamsutter, Wyoming",1169591845,GA,Wyoming
21506,"Wheatland, Wyoming",1176370621,GA,Wyoming
21507,"Worland, Wyoming",1166347917,GA,Wyoming
21508,"Wright, Wyoming",1166334449,GA,Wyoming


In [40]:
wikipedia_df['state'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

The next step is to access the Population Data.

In [41]:
# Read the population data
population_df = pd.read_excel('NST-EST2022-POP.xlsx')
population_df

,table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Annual Estimates of the Resident Population fo...,NaN,NaN,NaN,NaN
1,Geographic Area,"April 1, 2020 Estimates Base",Population Estimate (as of July 1),NaN,NaN
2,NaN,NaN,2020,2021.0,2022.0
3,United States,331449520,331511512,332031554.0,333287557.0
4,Northeast,57609156,57448898,57259257.0,57040406.0
...,...,...,...,...,...
61,Note: The estimates are developed from a base ...,NaN,NaN,NaN,NaN
62,Suggested Citation:,NaN,NaN,NaN,NaN
63,Annual Estimates of the Resident Population fo...,NaN,NaN,NaN,NaN
64,"Source: U.S. Census Bureau, Population Division",NaN,NaN,NaN,NaN


In [42]:
# Select the state and the 2022 population column
population_df = population_df.iloc[:,[0,-1]]

In [43]:
# Rename the columns and remove unnecessary rows
population_df.columns = ['state','population']
population_df = population_df.iloc[8:61,:].reset_index(drop=True)

In [44]:
population_df

,state,population
0,.Alabama,5074296.0
1,.Alaska,733583.0
2,.Arizona,7359197.0
3,.Arkansas,3045637.0
4,.California,39029342.0
5,.Colorado,5839926.0
6,.Connecticut,3626205.0
7,.Delaware,1018396.0
8,.District of Columbia,671803.0
9,.Florida,22244823.0


The state names need to be stripped off the '.' character and the non-state names like District of Columbia and Puerto Rico need to be dropped.

In [45]:
# Stripping the '.' and removing non-state rows
population_df['state'] = population_df['state'].str.lstrip('.')
non_states = ['District of Columbia', 'Puerto Rico']
population_df = population_df[~population_df['state'].isin(non_states)]
# Removing the last NaN row
population_df = population_df.iloc[:-1,:]
population_df = population_df.reset_index(drop=True)

In [46]:
population_df

,state,population
0,Alabama,5074296.0
1,Alaska,733583.0
2,Arizona,7359197.0
3,Arkansas,3045637.0
4,California,39029342.0
5,Colorado,5839926.0
6,Connecticut,3626205.0
7,Delaware,1018396.0
8,Florida,22244823.0
9,Georgia,10912876.0


Now we will access the Region-State mapping data for all the US states.

In [47]:
# Perform forward fill on region dataframe in the division column to fill up the empty rows
region_df = pd.read_excel('US States by Region - US Census Bureau.xlsx')
region_df['DIVISION'].fillna(method = 'ffill', inplace = True)
region_df

,REGION,DIVISION,STATE
0,Northeast,NaN,NaN
1,NaN,New England,NaN
2,NaN,New England,Connecticut
3,NaN,New England,Maine
4,NaN,New England,Massachusetts
...,...,...,...
58,NaN,Pacific,Alaska
59,NaN,Pacific,California
60,NaN,Pacific,Hawaii
61,NaN,Pacific,Oregon


In [48]:
# Remove unwanted columns and rows with null values
region_df = region_df.drop('REGION', axis=1)
region_df.dropna(inplace=True)
region_df = region_df.reset_index(drop=True)
# Rename the columns
region_df.columns = ['regional_division','state']

In [49]:
region_df

,regional_division,state
0,New England,Connecticut
1,New England,Maine
2,New England,Massachusetts
3,New England,New Hampshire
4,New England,Rhode Island
5,New England,Vermont
6,Middle Atlantic,New Jersey
7,Middle Atlantic,New York
8,Middle Atlantic,Pennsylvania
9,East North Central,Illinois


## 2. Data Preprocessing

Some preprocessing of the data will be necessary in terms of merging the wikipedia data and population data together. Both files have fields containing state names for just that purpose. The combined dataset also requires labeling each state with its US Census regional-division.

In [50]:
# Merge the population and wikipedia dataframe
merged_data1 = pd.merge(population_df, wikipedia_df, left_on='state', right_on='state', how='outer')
merged_data1.head(10)

,state,population,article_title,revision_id,article_quality
0,Alabama,5074296.0,"Abbeville, Alabama",1.171164e+09,C
1,Alabama,5074296.0,"Adamsville, Alabama",1.177621e+09,C
2,Alabama,5074296.0,"Addison, Alabama",1.168360e+09,C
3,Alabama,5074296.0,"Akron, Alabama",1.165910e+09,GA
4,Alabama,5074296.0,"Alabaster, Alabama",1.179140e+09,C
5,Alabama,5074296.0,"Albertville, Alabama",1.179199e+09,C
6,Alabama,5074296.0,"Alexander City, Alabama",1.179140e+09,GA
7,Alabama,5074296.0,"Aliceville, Alabama",1.167792e+09,GA
8,Alabama,5074296.0,"Allgood, Alabama",1.165910e+09,C
9,Alabama,5074296.0,"Altoona, Alabama",1.165910e+09,C


In [51]:
# Merge the above merged dataframe with the region dataframe
merged_data2 = pd.merge(region_df, merged_data1, left_on='state', right_on='state', how='outer')
merged_data2

,regional_division,state,population,article_title,revision_id,article_quality
0,New England,Connecticut,3626205.0,NaN,NaN,NaN
1,New England,Maine,1385340.0,"Abbot, Maine",1.171169e+09,Start
2,New England,Maine,1385340.0,"Acton, Maine",1.175249e+09,C
3,New England,Maine,1385340.0,"Addison, Maine",1.168360e+09,C
4,New England,Maine,1385340.0,"Albion, Maine",1.165910e+09,C
...,...,...,...,...,...,...
21507,Pacific,Washington,7785786.0,"Wilkeson, Washington",1.176401e+09,GA
21508,Pacific,Washington,7785786.0,"Wilson Creek, Washington",1.134836e+09,GA
21509,Pacific,Washington,7785786.0,"Winthrop, Washington",1.177588e+09,C
21510,Pacific,Washington,7785786.0,"Yacolt, Washington",1.175858e+09,C


In [52]:
# Rearrange the columns to get the desired schema
merged_data2 = merged_data2[['state', 'regional_division', 'population', 'article_title', 'revision_id'
                            , 'article_quality']]
merged_data2

,state,regional_division,population,article_title,revision_id,article_quality
0,Connecticut,New England,3626205.0,NaN,NaN,NaN
1,Maine,New England,1385340.0,"Abbot, Maine",1.171169e+09,Start
2,Maine,New England,1385340.0,"Acton, Maine",1.175249e+09,C
3,Maine,New England,1385340.0,"Addison, Maine",1.168360e+09,C
4,Maine,New England,1385340.0,"Albion, Maine",1.165910e+09,C
...,...,...,...,...,...,...
21507,Washington,Pacific,7785786.0,"Wilkeson, Washington",1.176401e+09,GA
21508,Washington,Pacific,7785786.0,"Wilson Creek, Washington",1.134836e+09,GA
21509,Washington,Pacific,7785786.0,"Winthrop, Washington",1.177588e+09,C
21510,Washington,Pacific,7785786.0,"Yacolt, Washington",1.175858e+09,C


In [53]:
# Identify areas missing data
null_rows = merged_data2[merged_data2.isnull().any(axis=1)]
null_rows

,state,regional_division,population,article_title,revision_id,article_quality
0,Connecticut,New England,3626205.0,NaN,NaN,NaN
12876,Nebraska,West North Central,1967923.0,NaN,NaN,NaN


In [54]:
missing_data_areas = null_rows['state'].to_list()
for item in missing_data_areas:
    print(item)

Connecticut
Nebraska


The two states - Connecticut and Nebraska seem to be missing the Wikipedia articles data. Thus, it looks like Wikipedia coverage is not as reliable since two whole states of the United States are missing completely, thereby introducing a coverage bias.

In [55]:
final_merged_data = merged_data2.dropna()
final_merged_data

,state,regional_division,population,article_title,revision_id,article_quality
1,Maine,New England,1385340.0,"Abbot, Maine",1.171169e+09,Start
2,Maine,New England,1385340.0,"Acton, Maine",1.175249e+09,C
3,Maine,New England,1385340.0,"Addison, Maine",1.168360e+09,C
4,Maine,New England,1385340.0,"Albion, Maine",1.165910e+09,C
5,Maine,New England,1385340.0,"Alexander, Maine",1.170294e+09,Start
...,...,...,...,...,...,...
21507,Washington,Pacific,7785786.0,"Wilkeson, Washington",1.176401e+09,GA
21508,Washington,Pacific,7785786.0,"Wilson Creek, Washington",1.134836e+09,GA
21509,Washington,Pacific,7785786.0,"Winthrop, Washington",1.177588e+09,C
21510,Washington,Pacific,7785786.0,"Yacolt, Washington",1.175858e+09,C


In [56]:
# Export the final merged data to csv
final_merged_data.to_csv('wp_scored_city_articles_by_state.csv', index=False)

## 3. Data Analysis

The analysis for this study will consist of calculating total-articles-per-population (a ratio representing the number of articles per person) and high-quality-articles-per-population (a ratio representing the number of high quality articles per person) on a state-by-state and divisional basis. All of these values are 'per capita' ratios.
For this analysis 'high quality' articles are considered as articles that ORES predicted would be in either the 'FA' (featured article) or 'GA' (good article) classes.

The data is then used to construct six following tables:
1. Top 10 US states by coverage: The 10 US states with the highest total articles per capita (in descending order) .
2. Bottom 10 US states by coverage: The 10 US states with the lowest total articles per capita (in ascending order) .
3. Top 10 US states by high quality: The 10 US states with the highest high quality articles per capita (in descending order) .
4. Bottom 10 US states by high quality: The 10 US states with the lowest high quality articles per capita (in ascending order).
5. Census divisions by total coverage: A rank ordered list of US census divisions (in descending order) by total articles per capita.
6. Census divisions by high quality coverage: Rank ordered list of US census divisions (in descending order) by high quality articles per capita.


### (a) Top 10 US States by Coverage

In [57]:
total_articles = final_merged_data.groupby('state')['article_title'].count()
state_pop = final_merged_data.groupby('state')['population'].mean()
articles_per_capita = total_articles/state_pop
articles_per_capita.sort_values(ascending=False,inplace=True)
state_coverage = articles_per_capita.to_frame()
state_coverage = state_coverage.rename(columns={0:'total-articles-per-capita'})
state_coverage.head(10)

,total-articles-per-capita
state,
Vermont,0.000507
North Dakota,0.000457
Maine,0.000349
South Dakota,0.000342
Iowa,0.000326
Alaska,0.000203
Pennsylvania,0.000197
Michigan,0.000177
Wyoming,0.000170


### (b) Bottom 10 US States by Coverage

In [58]:
articles_per_capita.sort_values(ascending=True,inplace=True)
state_coverage_bottom = articles_per_capita.to_frame()
state_coverage_bottom = state_coverage_bottom.rename(columns={0:'total-articles-per-capita'})
state_coverage_bottom.head(10)

,total-articles-per-capita
state,
North Carolina,0.000005
Nevada,0.000006
California,0.000012
Arizona,0.000012
Virginia,0.000015
Florida,0.000018
Oklahoma,0.000019
Kansas,0.000021
Maryland,0.000025


### (c) Top 10 US States by High Quality

In [59]:
final_merged_data['high_quality_articles'] = ((final_merged_data['article_quality'] == 'FA') | 
                                              (final_merged_data['article_quality'] == 'GA'))
total_high_quality = final_merged_data.groupby('state')['high_quality_articles'].sum()
high_quality_per_capita = total_high_quality/state_pop
high_quality_per_capita.sort_values(ascending=False,inplace=True)
high_quality = high_quality_per_capita.to_frame()
high_quality = high_quality.rename(columns={0:'high-quality-articles-per-capita'})
high_quality.head(10)

,high-quality-articles-per-capita
state,
Vermont,0.000070
Wyoming,0.000067
South Dakota,0.000062
West Virginia,0.000060
Montana,0.000049
New Hampshire,0.000045
Pennsylvania,0.000044
Missouri,0.000043
Alaska,0.000042


### (d) Bottom 10 US States by High Quality

In [60]:
high_quality_per_capita.sort_values(ascending=True,inplace=True)
high_quality_bottom = high_quality_per_capita.to_frame()
high_quality_bottom = high_quality_bottom.rename(columns={0:'high-quality-articles-per-capita'})
high_quality_bottom.head(10)

,high-quality-articles-per-capita
state,
North Carolina,0.000002
Virginia,0.000002
Nevada,0.000003
Arizona,0.000003
California,0.000004
Florida,0.000005
New York,0.000006
Maryland,0.000007
Kansas,0.000007


### (e) Census Divisions by Total Coverage

In [61]:
total_articles = final_merged_data.groupby('regional_division')['article_title'].count()
state_pop = final_merged_data.groupby(['regional_division','state'])[['population']].mean()
division_pop = state_pop.groupby('regional_division')['population'].sum()
articles_per_capita = total_articles/division_pop
articles_per_capita.sort_values(ascending=False,inplace=True)
division_coverage = articles_per_capita.to_frame()
division_coverage = division_coverage.rename(columns={0:'total-articles-per-capita'})
division_coverage

,total-articles-per-capita
regional_division,
West North Central,0.000181
New England,0.000125
East North Central,0.000101
Middle Atlantic,0.000090
East South Central,0.000078
West South Central,0.000050
Mountain,0.000047
South Atlantic,0.000028
Pacific,0.000024


### (f) Census Divisions by High Quality Coverage

In [62]:
total_high_quality = final_merged_data.groupby('regional_division')['high_quality_articles'].sum()
high_quality_per_capita = total_high_quality/division_pop
high_quality_per_capita.sort_values(ascending=False,inplace=True)
high_quality = high_quality_per_capita.to_frame()
high_quality = high_quality.rename(columns={0:'high-quality-articles-per-capita'})
high_quality

,high-quality-articles-per-capita
regional_division,
West North Central,0.000032
Middle Atlantic,0.000025
New England,0.000020
East South Central,0.000016
East North Central,0.000015
West South Central,0.000015
Mountain,0.000013
Pacific,0.000009
South Atlantic,0.000008
